In [1]:
import pandas as pd
import numpy as np
import unityneuro.render as urn

from ibllib.atlas import AllenAtlas
atlas = AllenAtlas(25)

from matplotlib import cm

viridis = cm.get_cmap('viridis', 100)

In [4]:
urn.setup()

(URN) connected to server
Login sent with ID: Dan


In [5]:
data = pd.read_csv('./data/repro_ephys/figure7_dataframe.csv')

In [6]:
data

,Unnamed: 0,cluster_ids,avg_fr,avg_fr_base,avg_fr_trial,avg_fr_rxn_time,avg_fr_post_stim,avg_fr_pre_move,avg_fr_pre_moveL,avg_fr_pre_moveR,...,y,z,p2t,amp,eid,pid,subject,probe,date,lab
0,0,48,1.154841,0.983302,1.465677,1.027660,1.465677,1.125541,1.239437,0.905797,...,-0.002250,-0.003708,0.466667,0.000049,824cf03d-4012-4ab1-b499-c83a92c5589e,c07d13ed-e387-4457-8e33-1d16aed3fd92,UCLA011,probe00,2021-07-20,churchlandlab_ucla
1,1,56,0.492202,0.000000,0.032468,0.034604,0.092764,0.024737,0.037559,0.000000,...,-0.002250,-0.003599,0.400000,0.000116,824cf03d-4012-4ab1-b499-c83a92c5589e,c07d13ed-e387-4457-8e33-1d16aed3fd92,UCLA011,probe00,2021-07-20,churchlandlab_ucla
2,2,60,0.467451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.002250,-0.003553,0.433333,0.000087,824cf03d-4012-4ab1-b499-c83a92c5589e,c07d13ed-e387-4457-8e33-1d16aed3fd92,UCLA011,probe00,2021-07-20,churchlandlab_ucla
3,3,76,2.828807,1.753247,5.273655,1.838325,3.228200,3.475572,3.643192,3.152174,...,-0.002250,-0.003354,0.466667,0.000050,824cf03d-4012-4ab1-b499-c83a92c5589e,c07d13ed-e387-4457-8e33-1d16aed3fd92,UCLA011,probe00,2021-07-20,churchlandlab_ucla
4,4,80,1.279031,0.584416,0.621521,0.353228,0.296846,0.420532,0.469484,0.326087,...,-0.002250,-0.003288,0.366667,0.000041,824cf03d-4012-4ab1-b499-c83a92c5589e,c07d13ed-e387-4457-8e33-1d16aed3fd92,UCLA011,probe00,2021-07-20,churchlandlab_ucla
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141,3141,162,11.769118,5.348837,19.418605,11.758606,20.891473,16.485788,16.300578,16.862745,...,-0.002549,-0.002080,-0.166667,0.000043,f140a2ec-fd49-4814-994a-fe3476f14e66,bf96f6d6-4726-4cfa-804a-bca8f9262721,NYU-47,probe00,2021-06-21,angelakilab
3142,3142,163,11.883847,10.368217,11.259690,10.230949,14.147287,11.782946,11.946050,11.450980,...,-0.002541,-0.002056,0.366667,0.000052,f140a2ec-fd49-4814-994a-fe3476f14e66,bf96f6d6-4726-4cfa-804a-bca8f9262721,NYU-47,probe00,2021-06-21,angelakilab
3143,3143,171,1.016561,0.658915,0.494186,0.815828,0.581395,0.956072,1.348748,0.156863,...,-0.002499,-0.001990,0.266667,0.000082,f140a2ec-fd49-4814-994a-fe3476f14e66,bf96f6d6-4726-4cfa-804a-bca8f9262721,NYU-47,probe00,2021-06-21,angelakilab
3144,3144,273,3.063816,2.965116,2.664729,3.479408,3.914729,3.255814,3.737958,2.274510,...,-0.002650,-0.002838,-0.200000,0.000034,f140a2ec-fd49-4814-994a-fe3476f14e66,bf96f6d6-4726-4cfa-804a-bca8f9262721,NYU-47,probe00,2021-06-21,angelakilab


In [7]:
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
def rgba_to_hex(rgba):
    return '#%02x%02x%02x%02x' % rgba

In [12]:
FR_thresh = 12.3
bregma = atlas.xyz2ccf([0,0,0])

neuronCount = 0
neuronNames = []
neuronLocs = {}
neuronColors = {}
neuronSizes = {}
neuronShapes = {}

for (i, row) in data.iterrows():
    ccf = atlas.xyz2ccf([row.x + np.random.normal()/200000, row.y, row.z + np.random.normal()/200000])
    
    if atlas.get_labels([row.x, row.y, row.z]) == 218:
        nn = 'n'+str(neuronCount)
        neuronCount+=1
        neuronNames.append(nn)
        neuronLocs[nn] = ccf.tolist()
        neuronSizes[nn] = 0.03 if row.avg_fr > FR_thresh else 0.015
        neuronShapes[nn] = 'sphere'
        
        # color
        color = viridis(row.avg_fr/20)
        rgba = (int(255*color[0]), int(255*color[1]), int(255*color[2]), int(255*color[3]))
        neuronColors[nn] = rgba_to_hex(rgba)

In [13]:
urn.clear()

urn.set_area_visibility({'8':False})
urn.set_area_material({'8':'transparent-unlit'})
urn.set_area_alpha({'8':0.1})

urn.set_area_visibility({'LP-l':True})
urn.set_area_material({'LP-l':'transparent-lit'})
urn.set_area_alpha({'LP-l':0.25})

urn.create_neurons(neuronNames)
urn.set_neuron_positions(neuronLocs)
urn.set_neuron_sizes(neuronSizes)
urn.set_neuron_shapes(neuronShapes)
urn.set_neuron_colors(neuronColors)

urn.set_camera_target_area('LP-l')